<a href="https://colab.research.google.com/github/2018007956/HYU-Capston-Project/blob/main/Inpainting.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [12]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# import os
# a = os.listdir('./House_Room_Dataset/Bathroom/')
# b = os.listdir('./House_Room_Dataset/Bedroom/')
# c = os.listdir('./House_Room_Dataset/Livingroom/')
# d = os.listdir('./House_Room_Dataset/Dinning/')
# e = os.listdir('./House_Room_Dataset/Kitchen/')
# print(len(a),len(b),len(c),len(d),len(e))
# print(len(a)+len(b)+len(c)+len(d)+len(e))

606 1248 1273 1158 965
5250


In [ ]:
# !pip install torch==1.8.0 torchvision==0.9.0
# !pip install torchtext==0.9.0
# !pip install pytorch_lightning

# Yolo

In [ ]:
import cv2
import torch
from PIL import Image
import numpy as np
import pandas as pd

In [ ]:
# !git clone https://github.com/ultralytics/yolov5
# %cd yolov5

In [ ]:
%cd '/content/drive/MyDrive/2022_2proj/yolov5'

/content/drive/MyDrive/2022_2proj/yolov5


In [ ]:
# !pip install -r https://raw.githubusercontent.com/ultralytics/yolov5/master/requirements.txt

In [ ]:
model = torch.hub.load('ultralytics/yolov5', 'yolov5s')

Using cache found in /root/.cache/torch/hub/ultralytics_yolov5_master
requirements: YOLOv5 requirement "torchvision>=0.8.1" not found, attempting AutoUpdate...
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached torch-1.8.0-cp38-cp38-manylinux1_x86_64.whl (735.5 MB)
  Attempting uninstall: torch
    Found existing installation: torch 1.13.0
    Uninstalling torch-1.13.0:
      Successfully uninstalled torch-1.13.0

requirements: 1 package updated per /root/.cache/torch/hub/ultralytics_yolov5_master/requirements.txt
requirements: ⚠️ Restart runtime or rerun command for updates to take effect

YOLOv5 🚀 2022-12-1 Python-3.8.15 torch-1.13.0+cu117 CUDA:0 (Tesla T4, 15110MiB)

Fusing layers... 
YOLOv5s summary: 213 layers, 7225885 parameters, 0 gradients
Adding AutoShape... 


In [ ]:
im = cv2.imread('../House_Room_Dataset/Bedroom/bed_11.jpg')[..., ::-1] # BGR to RGB
res = model(im)
res.print()
res.show()
res.xyxy[0]
res.pandas().xyxy[0]

RuntimeError: ignored

In [ ]:
# name 이 bed가 아닌 것만 골라내기
df = res.pandas().xyxy[0]

hold_list = ['bench', 'chair', 'couch','bed','dining table', 'toilet','tv','microwave','oven',
        'toaster','sink','refrigerator','clock','vase','hair drier','toothbrush']
df = pd.DataFrame([x for i,x in df.iterrows() if x['name'] not in hold_list])
df

In [ ]:
patches = []
for idx, row in df.iterrows():
  print(row['xmin'],row['xmax'],row['ymin'],row['ymax'])
  crop_img = im[int(row['xmin']):int(row['xmax']),int(row['ymin']):int(row['ymax'])]
  patches.append(crop_img)

In [ ]:
Image.fromarray(patches[0])

In [ ]:
# Make mask Images

x = df.loc[df.index[0]]
mask_im = im
# mask_im[int(x['xmin']):int(x['xmax']),int(x['ymin']):int(x['ymax'])]=0
for idx,x in df.iterrows():
  mask_im[int(x['xmin']):int(x['xmax']),int(x['ymin']):int(x['ymax'])]=0


mask = Image.fromarray(mask_im)
mask.save('./configs/prediction/bed_11_mask001.jpg')###/default.yaml') ##########

# LaMa

In [2]:
%cd /content/drive/MyDrive/2022_2proj

/content/drive/MyDrive/2022_2proj


In [3]:
# !git clone https://github.com/saic-mdal/lama.git

%cd lama
# !pip install -r requirements.txt 

/content/drive/MyDrive/2022_2proj/lama


In [ ]:
# !export TORCH_HOME=$(pwd) && export PYTHONPATH=$(pwd)

In [ ]:
# # Download pre-trained models
# !pip3 install wldhx.yadisk-direct

# # best model
# !curl -L $(yadisk-direct https://disk.yandex.ru/d/ouP6l8VJ0HpMZg) -o big-lama.zip
# !unzip big-lama.zip

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:--  0:00:02 --:--:--     0
100  363M    0  363M    0     0  8476k      0 --:--:--  0:00:43 --:--:-- 10.1M
Archive:  big-lama.zip
replace big-lama/config.yaml? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: big-lama/config.yaml    
replace big-lama/models/best.ckpt? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: big-lama/models/best.ckpt  


In [4]:
!pip install pytorch_lightning 
!pip install kornia --no-dependencies
!pip install hydra
!pip install omegaconf
!pip install webdataset
!pip install hydra-core --upgrade

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 800.3/800.3 KB 32.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 512.4/512.4 KB 46.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 125.4/125.4 KB 17.8 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 569.1/569.1 KB 11.7 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.4/82.4 KB 8.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for hydra: filename=Hydra-2.5-cp38-cp38-linux_x86_64.whl size=253707 sha256=ca24340fd11a17488c1ecbdbdb3be3c63d0c9049d41b895cb3d13964b2a8da4a
  Stored in directory: /root/.cache/pip/wheels/d0/25/3a/15ed76174a2578d1bdd53d62fabf876273df191c3f660

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 KB 7.0 MB/s eta 0:00:00


### Download pre-trained models

In [ ]:
# !pip3 install wldhx.yadisk-direct
# !curl -L $(yadisk-direct https://disk.yandex.ru/d/ouP6l8VJ0HpMZg) -o big-lama.zip
# !unzip big-lama.zip

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:--  0:00:01 --:--:--     0
100  363M    0  363M    0     0  9816k      0 --:--:--  0:00:37 --:--:--  9.9M


In [14]:
!python bin/predict.py model.path=$(pwd)/big-lama indir=$(pwd)/data_for_prediction outdir=$(pwd)/output

Detectron v2 is not installed
bin/predict.py:38: UserWarning: 
The version_base parameter is not specified.
Please specify a compatability version level, or None.
Will assume defaults for version 1.1
  @hydra.main(config_path='../configs/prediction', config_name='default.yaml')
/usr/local/lib/python3.8/dist-packages/hydra/_internal/hydra.py:119: UserWarning: Future Hydra versions will no longer change working directory at job runtime by default.
See https://hydra.cc/docs/1.2/upgrades/1.1_to_1.2/changes_to_job_working_dir/ for more information.
  ret = run_job(
[2023-01-05 07:16:44,001][saicinpainting.utils][WARNING] - Setting signal 10 handler <function print_traceback_handler at 0x7f13e8877f70>
[2023-01-05 07:16:44,031][root][INFO] - Make training model default
[2023-01-05 07:16:44,031][saicinpainting.training.trainers.base][INFO] - BaseInpaintingTrainingModule init called
[2023-01-05 07:16:44,032][root][INFO] - Make generator ffc_resnet
[2023-01-05 07:16:44,499][saicinpainting.traini

In [ ]:
!pwd

/content/drive/MyDrive/2022_2proj/lama


In [ ]:
# image size와 같도록 mask resize
import cv2
path = '/content/drive/MyDrive/2022_2proj/lama/data_for_prediction/'

i = cv2.imread(path+'room_2.png')
image_shape = i.shape
print(image_shape)
m = cv2.imread(path+'room_2_mask.png')
print(m.shape)
resize_m = cv2.resize(m, (image_shape[1],image_shape[0]))
print(resize_m.shape)
cv2.imwrite(path+'room_2_mask.png',resize_m)

(640, 960, 3)
(448, 640, 3)
(640, 960, 3)


True

In [13]:
import numpy as np
import cv2
import os
from glob import glob
mask_lst = glob('/content/drive/MyDrive/2022_2proj/lama/data_for_prediction/*_mask.png')

for i in mask_lst:
  img1 = cv2.imread(i, cv2.IMREAD_GRAYSCALE)
  k = cv2.getStructuringElement(cv2.MORPH_RECT, (11,11))
  dst = cv2.dilate(img1, k)

  cv2.imwrite(i[:-4]+"_Dilation.png", dst)